In [ ]:
print("ok")

1. Import Dependecies

In [ ]:
# Import dependencies
import os
from langchain_community.vectorstores import Pinecone
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
# from langchain.vectorstores import Pinecone 
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from ctransformers import AutoModelForCausalLM
from dotenv import load_dotenv
import pypdf

2. Pinecone cluster

In [ ]:
load_dotenv()

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")

3. Doc Loader

In [ ]:
# Extract Data from the PDF
# https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents



4. Extract data

In [ ]:
extracted_data = load_pdf("../data/")

In [ ]:
# extracted_data
print(f"length of extracted data: {len(extracted_data)}")

5. Text splitting/ chunking

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print(f"length of the text chunks: {len(text_chunks)}")


6. Download embedding model

In [ ]:
def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
# call the download_huggingface_embedding function
embeddings = download_huggingface_embedding()

In [ ]:
embeddings

6.2 Check if embedding performs as expected

In [ ]:
# embedding_result = embeddings.embed_query("What is the capital of France?")
# print(f"length of the embeddings: {len(embedding_result)}")
# It performs as expected :)

7. Creat pincone vector for each chunk and store them

In [ ]:
# Initialize the Pinecone vector store
pc= Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# Create a Pinecone index
index_name = "medbot"

# Create a Pinecone index
index = pc.Index("medbot")

# Create embeddings for the text chunks and store them in the Pinecone index
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

8. Generate Ranked Results

In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

query="What is an inventory"

docs = docsearch.similarity_search(query, k=3)

print("Results", docs[0].page_content)

9. Define a prompt templete

In [ ]:
prompt_template='''
Use the following pices of information to answer the question.
If you do not know the answer, please say "Regret, I do not know!", do not guess or make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Answer:
'''

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

10. Define the LLM

In [30]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_file="ggmlv3.q4_0.bin",                  
                  model_type="llama",
                  congif={"max_tokens": 512,
                  "temparatue":0.8},
                  )

In [33]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x7f5a553962c0>, model='../model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', model_file='ggmlv3.q4_0.bin')

In [32]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chin_type="stuff",
    retriver=docsearch.as_retriever(),
    return_source_document=True,
    chain_type_kwargs=chain_type_kwargs,
)

ValidationError: 1 validation error for LLMChain
llm
  Can't instantiate abstract class BaseLanguageModel with abstract methods agenerate_prompt, apredict, apredict_messages, generate_prompt, predict, predict_messages (type=type_error)